This notebook contains example circuit classes.

The three circuits *Not*, *Or*, and *And* are used to build progressively more complicated circuits culminating in a four bit binary adder.

In [50]:
class Connector():
    def __init__(self, owner, name, activates=0, monitor=0):
        self.value = None
        self.owner = owner
        self.name = name
        self.monitor = monitor
        self.connects = []
        self.activates = activates
        
    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
        for input in inputs:
            self.connects.append(input)
            
    def set(self, value):
        if self.value == value:
            return
        self.value = value
        if self.activates:
            self.owner.evaluate()
        if self.monitor:
            print('Connector {}-{} set to {}'.format(self.owner.name,
                                                    self.name,
                                                    self.value))
        for con in self.connects:
            con.set(value)
            

    

In [54]:
class LC:
    def __init__(self, name):
        self.name = name
        
    def evaluate(self):
        return
    
class Not(LC):
    def __init__(self, name):
        super().__init__( name)
        self.A = Connector(self, 'A', activates=1)
        self.B = Connector(self, 'B')
        
    def evaluate(self):
        self.B.set(not self.A.value)
        
class Gate2(LC):
    def __init__(self, name):
        super().__init__( name)
        
        self.A = Connector(self, 'A', activates=1)
        self.B = Connector(self, 'B', activates=1)
        self.C = Connector(self, 'C')
        
class AndGate(Gate2):
    def __init__(self, name):
        super().__init__( name)
        
    def evaluate(self):
        self.C.set(self.A.value and self.B.value)
        
class Or(Gate2):
    def __init__(self, name):
        super().__init__( name)
        
    def evaluate(self):
        self.C.set(self.A.value or self.B.value)
        
class Xor(Gate2):
    def __init__(self, name):
        super().__init__(name)
        
        self.A1 = AndGate('A1')
        self.A2 = AndGate('A2')
        self.N1 = Not('N1')
        self.N2 = Not('N2')
        self.OR1 = Or('OR1')
        
        self.A.connect([self.A1.A, self.N2.A])
        self.B.connect([self.N1.A, self.A2.A])
        
        self.N1.B.connect([self.A1.B])
        self.N2.B.connect([self.A2.B])
        
        self.A1.C.connect([self.OR1.A])
        self.A2.C.connect([self.OR1.B])
        
        self.OR1.C.connect([self.C])

        



In [77]:
class HalfAdder(LC):
    def __init__(self, name):
        super().__init__(name)
        
        self.A = Connector(self, 'A', 1)
        self.B = Connector(self, 'B', 1)
        self.S = Connector(self, 'S')
        self.C = Connector(self, 'C')
        
        self.XOR1 = Xor('XOR1')
        self.A1 = AndGate('A1')
        
        self.A.connect([self.A1.A, self.XOR1.A])
        self.B.connect([self.A1.B, self.XOR1.B])
        
        self.XOR1.C.connect([self.S])
        self.A1.C.connect([self.C])
        
class FullAdder(LC):
    def __init__(self, name):
        super().__init__(name)
        
        self.A = Connector(self, 'A', 1)
        self.B = Connector(self, 'B', 1)
        self.Cin = Connector(self, 'Cin', 1)
        self.S = Connector(self, 'S')
        self.S.monitor=1
        self.Cout = Connector(self, 'Cout')
        self.Cout.monitor=1
        
        self.Or1 = Or('OR1')
        self.HA1 = HalfAdder('HA1')
        self.HA2 = HalfAdder('HA2')
        
        self.A.connect([self.HA1.A])
        self.B.connect([self.HA1.B])
        self.Cin.connect([self.HA2.A])
        self.HA1.S.connect([self.HA2.B])
        
        self.HA1.C.connect([self.Or1.A])
        self.HA2.C.connect([self.Or1.B])
        
        self.HA2.S.connect([self.S])
        
        self.Or1.C.connect([self.Cout])
        
    

In [78]:
ha = FullAdder('H1')
ha.A.set(1)
ha.B.set(1)
ha.Cin.set(1)

Connector H1-Cout set to 1
Connector H1-S set to False
Connector H1-S set to True


In [81]:
def bit(x, bit):
    return x[bit] == '1'

In [86]:
def test4Bit(a,b):
    F0 = FullAdder('F0')
    F1 = FullAdder('F1')
    F0.Cout.connect(F1.Cin)
    F2 = FullAdder('F2')
    F1.Cout.connect(F2.Cin)
    F3 = FullAdder('F3')
    F2.Cout.connect(F3.Cin)
    
    F0.Cin.set(0)
    F0.A.set(bit(a, 3))
    F0.B.set(bit(b, 3))
    F1.A.set(bit(a, 2))
    F1.B.set(bit(b, 2))
    F2.A.set(bit(a, 1))
    F2.B.set(bit(b, 1))
    F3.A.set(bit(a, 0))
    F3.B.set(bit(b, 0))
    
    print(int(F3.Cout.value), int(F3.S.value), int(F2.S.value),
          int(F1.S.value), int(F0.S.value))
    
    
    

In [87]:
test4Bit('0011', '0110')

Connector F0-Cout set to 0
Connector F1-Cout set to 0
Connector F2-Cout set to 0
Connector F3-Cout set to 0
Connector F0-S set to True
Connector F1-Cout set to True
Connector F2-Cout set to None
Connector F3-Cout set to None
Connector F1-S set to False
Connector F2-Cout set to True
Connector F2-S set to False
Connector F3-Cout set to False
Connector F3-S set to True
0 1 0 0 1
